# When Explanations Lie: Why Modified BP Attribution fails

This is the code to the paper [].



In [ ]:
# uncomment to install install packages
#!pip install innvestigate seaborn tqdm
#!pip install tensorflow-gpu==1.13.1

In [ ]:
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow
import tensorflow as tf

import innvestigate
import matplotlib.pyplot as plt

import numpy as np
import PIL 
import copy
import contextlib

import imp
import numpy as np
import os

from skimage.measure import compare_ssim 
import pickle
from collections import OrderedDict
from IPython.display import IFrame, display

import keras
import keras.backend
import keras.models


import innvestigate
import innvestigate.applications.imagenet
import innvestigate.utils as iutils
import innvestigate.utils as iutils
import innvestigate.utils.visualizations as ivis
from innvestigate.analyzer.relevance_based.relevance_analyzer import LRP
from innvestigate.analyzer.base import AnalyzerNetworkBase, ReverseAnalyzerBase
from innvestigate.analyzer.deeptaylor import DeepTaylor

import time
import tqdm

import seaborn as sns

import itertools
import matplotlib as mpl

from tensorflow.python.client import device_lib
from scipy.special import softmax

from when_explanations_lie import *
import json

In [ ]:
def _prepare_model(self, model):
    return super(DeepTaylor, self)._prepare_model(model)

# otherwise DTD does not work on negative outputs
DeepTaylor._prepare_model = _prepare_model

In [ ]:
device_lib.list_local_devices()

In [ ]:
# path to imagenet validation
#imagenet_val_dir = "/home/leonsixt/tmp/imagenet/imagenet-raw/validation"
imagenet_val_dir = "/mnt/ssd/data/imagenet/imagenet-raw/validation"
# path to examplary image
ex_image_path = "n01534433/ILSVRC2012_val_00015410.JPEG"
# number of images to run the evaluation
n_selected_imgs = 200

model_names = ['resnet50', 'vgg16']

assert os.path.exists(imagenet_val_dir)
os.makedirs('figures', exist_ok=True)

In [ ]:

output_shapes = {}
model, innv_net, color_conversion = load_model('vgg16')
ex_image_vgg, ex_target, val_images, selected_img_idxs = load_val_images(
    innv_net, imagenet_val_dir, ex_image_path, n_selected_imgs)

output_shapes['vgg16'] = get_output_shapes(model)

keras.backend.clear_session()
model, innv_net, color_conversion = load_model('resnet50')
ex_image, ex_target, val_images, selected_img_idxs = load_val_images(
    innv_net, imagenet_val_dir, ex_image_path, n_selected_imgs)

output_shapes['resnet50'] = get_output_shapes(model)

assert ((ex_image - ex_image_vgg) == 0).all()

nice_layer_names = get_nice_layer_names(model)


In [ ]:
correct = []
for img, target in tqdm.tqdm_notebook(val_images):
    logits = model.predict(img)
    top1 = logits.argmax()
    correct.append(top1 == target)
    
print("Top-1 accuracy: ", np.mean(correct))

In [ ]:
n_layers = {'vgg16': 23, 'resnet50': 177}

randomization_layers = {
    'vgg16': ["conv1_1", "conv2_1",  "conv3_1", "conv4_1",  "conv4_3",  "conv5_1",
        "conv5_3", "fc1", "fc3"],
    'resnet50': ['conv1', 'block2_2', 'block3_2', 'block4_2', 'block5_2', 'dense'],
}

replacement_layers = {
    'vgg16':  ['fc3', 'fc1', 'conv4_3', 'conv3_3', 'conv2_2'],
    'resnet50': ['dense', 'block5_1', 'block4_2', 'block3_4', 'block3_2', 'block2_2'],
}

In [ ]:

with open('imagenet_labels.json', 'r') as f:
    idx_to_label =  json.load(f)

In [ ]:
cat_dog = PIL.Image.open('images/cat_dog_224.jpg')
cat  = PIL.Image.open('images/cat_224.jpg')
dog  = PIL.Image.open('images/dog_224.jpg')

zebra_elephant = PIL.Image.open('images/zebra_elephant_224.jpg')
zebra  = PIL.Image.open('images/zebra_224.jpg')
elephant  = PIL.Image.open('images/elephant_224.jpg')

two_class_pil_imgs = [
    # Persian cat, King Charles Spaniel 
    (cat_dog, cat, dog, 283, 156),  
    # Zebra, African bush elephant
    (zebra_elephant, zebra, elephant, 340, 386)
]
two_class_imgs = [(
    preprocess(np.array(a_and_b).astype(np.float), innv_net)[None], 
    preprocess(np.array(a).astype(np.float), innv_net)[None], 
    preprocess(np.array(b, ).astype(np.float), innv_net)[None], 
    a_cls,
    b_cls
) for a_and_b, a, b, a_cls, b_cls in two_class_pil_imgs]

In [ ]:
for ab, a, b, a_cls, b_cls in two_class_imgs:
    logit_ab = model.predict(ab)[0]
    logit_a = model.predict(a)[0]
    logit_b = model.predict(b)[0]
    
    idx_ab = np.argsort(logit_ab)[::-1]
    idx_a = np.argsort(logit_a)[::-1]
    idx_b = np.argsort(logit_b)[::-1]
    
    
    prob_a = softmax(logit_a, -1)
    prob_b = softmax(logit_b, -1)
    
    for top_class in idx_a[:5]:
        print(prob_a[top_class], top_class, idx_to_label[int(top_class)])
        #print()
        
    print()
        
    for top_class in idx_b[:5]:
        print(prob_b[top_class], top_class, idx_to_label[int(top_class)])
        
    print()
    print("-" * 60)
    print()

In [ ]:
def normalize_neg(x):
    vmax = np.percentile(x, 99)
    vmin = np.percentile(x, 1)
    x_pos = x * (x > 0)
    x_neg = x * (x < 0)
    x_pos = x_pos / vmax
    x_neg = - x_neg / vmin
    return np.abs(np.clip(x_pos + x_neg, -1, 1))


In [ ]:
for model_name in model_names:
    print(model_name)
    keras.backend.clear_session()
    model, innv_net, color_conversion = load_model(model_name)
    ex_image_vgg, ex_target, val_images, selected_img_idxs = load_val_images(
        innv_net, imagenet_val_dir, ex_image_path, n_selected_imgs)

    analyser = innvestigate.create_analyzer('deep_taylor', model, neuron_selection_mode="index")
    
    for i, (ab, a, b, a_cls, b_cls) in enumerate(two_class_imgs):
        ab_explain_a = analyser.analyze(ab, a_cls)
        ab_explain_b = analyser.analyze(ab, b_cls)
        ab_explain_random = analyser.analyze(ab, np.random.choice([i for i in range(1000) if i not in [a_cls, b_cls]]))

        fig, axes = plt.subplots(1, 4, figsize=(15, 4))


        axes[0].imshow(two_class_pil_imgs[i][0])
        axes[0].set_title("Image")
        hmap_a = normalize_neg(ab_explain_a[0].sum(-1))
        hmap_b = normalize_neg(ab_explain_b[0].sum(-1))

        axes[1].imshow(hmap_a, vmin=-1, vmax=1, cmap='seismic')
        axes[1].set_title(idx_to_label[a_cls])

        axes[2].imshow(hmap_b, vmin=-1, vmax=1, cmap='seismic')
        axes[2].set_title(idx_to_label[b_cls])

        axes[3].imshow(hmap_a - hmap_b, vmin=-1, vmax=1, cmap='seismic')
        axes[3].set_title('"{}" - "{}"'.format(idx_to_label[a_cls], idx_to_label[b_cls]))

        print(np.abs(hmap_a - hmap_b).mean())
        print(np.sqrt(((hmap_a - hmap_b)**2).mean()))

        for ax in axes:
            ax.set_xticks([])
            ax.set_yticks([])
        plt.show()

        plt.imshow((ab_explain_a - ab_explain_b)[0].sum(-1), cmap='seismic')
        plt.colorbar()

        plt.xticks([])
        plt.yticks([])
        plt.show()

        #plt.imshow(normalize_neg(ab_explain_random[0].sum(-1)), vmin=-1, vmax=1, cmap='seismic')
        #plt.show()


In [ ]:
for model_name in ['vgg16']:
    print(model_name)
    keras.backend.clear_session()
    model, innv_net, color_conversion = load_model(model_name)
    ex_image_vgg, ex_target, val_images, selected_img_idxs = load_val_images(
        innv_net, imagenet_val_dir, ex_image_path, n_selected_imgs)

    analyser = innvestigate.create_analyzer('deep_taylor', model, neuron_selection_mode="index")
    
    for i, (ab, a, b, a_cls, b_cls) in enumerate(two_class_imgs):
        ab_explain_a = analyser.analyze(ab, a_cls)
        ab_explain_b = analyser.analyze(ab, b_cls)
        ab_explain_random = analyser.analyze(ab, np.random.choice([i for i in range(1000) if i not in [a_cls, b_cls]]))

        fig, axes = plt.subplots(1, 2, figsize=(10, 5))


        axes[0].imshow(two_class_pil_imgs[i][0])
        axes[0].set_title("Image")
        hmap_a = normalize_neg(ab_explain_a[0].sum(-1))
        hmap_b = normalize_neg(ab_explain_b[0].sum(-1))

        axes[1].imshow(hmap_a, vmin=-1, vmax=1, cmap='seismic')
        axes[1].set_title("Why did you say: " + idx_to_label[a_cls] + "?")
        for ax in axes:
            ax.set_xticks([])
            ax.set_yticks([])
        plt.show()
        
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        axes[0].imshow(two_class_pil_imgs[i][0])
        axes[0].set_title("Image")
        axes[1].imshow(hmap_b, vmin=-1, vmax=1, cmap='seismic')
        axes[1].set_title("Why did you say: " + idx_to_label[b_cls] + "?")

        print(np.abs(hmap_a - hmap_b).mean())
        print(np.sqrt(((hmap_a - hmap_b)**2).mean()))

        for ax in axes:
            ax.set_xticks([])
            ax.set_yticks([])
        plt.show()

        plt.imshow((ab_explain_a - ab_explain_b)[0].sum(-1), cmap='seismic')
        plt.colorbar()

        plt.xticks([])
        plt.yticks([])
        plt.show()

        #plt.imshow(normalize_neg(ab_explain_random[0].sum(-1)), vmin=-1, vmax=1, cmap='seismic')
        #plt.show()


In [ ]:
zebra_test = preprocess(load_image("images/zebra_224.jpg", 224), innv_net)

In [ ]:
np.abs(zebra_test - two_class_imgs[1][1]).mean()

In [ ]:
# path to imagenet validation
# imagenet_val_dir = "/home/leonsixt/tmp/imagenet/imagenet-raw/validation"
imagenet_val_dir = "/mnt/ssd/data/imagenet/imagenet-raw/validation"
# path to examplary image
ex_image_path = "n01534433/ILSVRC2012_val_00015410.JPEG"
# number of images to run the evaluation
n_selected_imgs = 200

model_names = ['resnet50', 'vgg16']

assert os.path.exists(imagenet_val_dir)
os.makedirs('figures', exist_ok=True)

In [ ]:
model, innv_net, color_conversion = load_model('vgg16')
ex_image_vgg, ex_target, val_images, selected_img_idxs = load_val_images(
    innv_net, imagenet_val_dir, ex_image_path, n_selected_imgs)

In [ ]:
ex_img, ex_target = val_images[0]

In [ ]:
dtd = innvestigate.create_analyzer('deep_taylor.bounded', model, low=ex_img.min(), high=ex_img.max())

In [ ]:
hmap = dtd.analyze(ex_img)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))


axes[0].imshow(norm_image(ex_img[0]))
axes[0].set_title("Image")
hmap_a = normalize_neg(hmap[0].sum(-1))

axes[1].imshow(hmap_a, vmin=-1, vmax=1, cmap='seismic')
axes[1].set_title("Why did you say: " + idx_to_label[ex_target] + "?")
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
plt.show()

In [ ]:
for top_class in idx[0][::-1][:20]:
    print(probs[0, top_class])
    print(top_class)
    print(idx_to_label[int(top_class)])

In [ ]:
PIL.Image.fromarray((255*norm_image(ex_img[0])).astype(np.uint8), 'RGB')

In [ ]:
input_range = (ex_image.min(), ex_image.max())
analysers = get_analyser_params(input_range)

attr_names = [n for (n, _, _, _, _) in analysers]
    
hmap_postprocesisng = {
    n: lambda x: heatmap_postprocess(post_name, x) for n, _, post_name, _, _ in analysers
}

In [ ]:
for i, name in enumerate(attr_names):
    style = mpl_styles[name]
    plt.plot(np.arange(10), [20-i] * 10, 
             #markersize=5,
             label=name + " m=" + style['marker'], **style)
    
plt.legend(bbox_to_anchor=(1, 1))

## Guided BP & Deconv

In [ ]:
## GUIDED

In [ ]:
for attr_name in ['gradient', 'guided_backprop']:
    grad_analyser = innvestigate.create_analyzer(attr_name, model, reverse_keep_tensors=True)
    
    for img, _ in val_images:
        hmap = grad_analyser.analyze(img)

        plots = {
            #'abs.mean': [],
            #'abs.median': [],
            'ratio_negative': [],
            'ratio_positive': [],
            'ratio_zeros': [],
        }
        for (idx, val) in grad_analyser._reversed_tensors[1:]:
            #print(idx, val.shape)
            #print(idx, val.mean())
            #plots['abs.mean'].append(np.abs(val).mean())
            #plots['abs.median'].append(np.abs(val).mean())
            plots['ratio_zeros'].append((val == 0).sum() / val.size)
            plots['ratio_negative'].append((val < 0).sum() / val.size)
            plots['ratio_positive'].append((val > 0).sum() / val.size)


        plt.figure(figsize=(20, 5))
        for name, values in plots.items():
            values = [values[i] for i in nice_layer_names[model_name].keys()]
            plt.plot(values[::-1], label=name)
        plt.ylim(0, 1)
        xticks = list(nice_layer_names[model_name].values())
        plt.xticks(np.arange(len(xticks)), xticks[::-1], rotation=90)
        plt.legend()
        plt.title(attr_name)
        plt.show()

In [ ]:
model, innv_net, color_conversion = load_model('vgg16')

In [ ]:
w, b = model.layers[-2].get_weights()

In [ ]:
w.shape

In [ ]:
wsq = w

In [ ]:
ev = np.linalg.eigvals(wsq.T)

In [ ]:
plt.plot(ev[np.argsort(ev)])

In [ ]:
# test guided bp
#  1. get grads
#  2. compute max(grad)
#  3. measure ssim before and after
#  4. backprop further
#  5. ....

In [ ]:
raise

In [ ]:

plt.figure(figsize=(15, 5))
for name, values in plots.items():
    plt.plot(values, label=name)
plt.ylim(0, 1)
plt.legend()
    #plt.show()

## Sanity Checks: Random Parameters & Logit Check

In [ ]:
model_cascading, _, _ = load_model('resnet50')
model_random, _, _ = load_model('resnet50', load_weights=False)
model_cascading.set_weights(model.get_weights())
out = model.predict(ex_image)
out_cascading = model_cascading.predict(ex_image)
print("mean-l1 distance of the outputs of the trained model and when weights are from trained model [should be 0]:", np.abs(out_cascading - out).mean())

n_layers = len(model_random.layers)
copy_weights(model_cascading, model_random, range(n_layers - 3, n_layers))

out = model.predict(ex_image)
out_cascading = model_cascading.predict(ex_image)
print( "mean-l1 distance of the outputs of the trained model when the last 2 layers are random [should not be 0]:", np.abs(out_cascading - out).mean())

In [ ]:
analysers[0]

In [ ]:
analysers[6:]

In [ ]:
nice_layer_names

In [ ]:
# heatmaps are saved in those dicts
hmap_original = OrderedDict()
hmap_random_weights = OrderedDict()
hmap_random_target = OrderedDict()

recreate_analyser = False
for model_name in tqdm.tqdm_notebook(model_names[::-1]):
    keras.backend.clear_session()
    
    model, innv_net, _ = load_model(model_name)
    model_cascading, _, _ = load_model(model_name)
    model_random, _, _ = load_model(model_name, load_weights=False)
    model_cascading.set_weights(model.get_weights())
    get_layer_idx = lambda layer_name: get_layer_idx_full(
        model_name, nice_layer_names, layer_name)
    
    for attr_name, innv_name, _, excludes, analyser_kwargs in tqdm.tqdm_notebook(
        analysers[:6], desc=model_name):
        
        if "exclude_" + model_name in excludes:
            continue
        if innv_name == 'pattern.attribution':
            analyser_kwargs['patterns'] = innv_net['patterns']

        cascading_heatmaps = {}
        cascading_outputs = {}
        model_cascading.set_weights(model.get_weights())

        original_idx = len(model.layers)

        analyzer_cascading = innvestigate.create_analyzer(
            innv_name, model_cascading, 
            neuron_selection_mode="index", **analyser_kwargs)

        for img_idx, (img_pp, target) in zip(selected_img_idxs, val_images):
            random_target = get_random_target(target)
            hmap_random_target[model_name, attr_name, img_idx] = (
                random_target, analyzer_cascading.analyze(img_pp, neuron_selection=random_target)[0])
        selected_layers = [('original', original_idx)] +  [
            (name, get_layer_idx(name)) 
             for name in randomization_layers[model_name][::-1]
        ]
        for layer_name, layer_idx in tqdm.tqdm_notebook(selected_layers, desc=attr_name):
            copy_weights(model_cascading, model_random, range(layer_idx, original_idx))
            if recreate_analyser:
                analyzer_cascading = create_analyzer(
                    analyser, model_cascading, 
                    neuron_selection_mode="index",  **analyser_kwargs)

            for img_idx, (img_pp, target) in zip(selected_img_idxs, val_images):
                hmap = analyzer_cascading.analyze(img_pp, neuron_selection=target)[0]
                if layer_idx == original_idx:
                    hmap_original[model_name, attr_name, img_idx] = hmap
                else:
                    hmap_random_weights[model_name, attr_name, img_idx, layer_idx] =  hmap

In [ ]:
dump_heatmaps = False

if dump_heatmaps:
    with open('heatmaps_v4.pickle', 'wb') as f:
        pickle.dump((hmap_original, hmap_random_weights, hmap_random_target, analysers), f)

In [ ]:
# print size of dump
! ls -lh 'heatmaps.pickle'

In [ ]:
load_heatmaps = False
if load_heatmaps:
    with open('heatmaps.pickle', 'rb') as f:
        hmap_original, hmap_random_weights, hmap_random_target, analysers = pickle.load(f)

In [ ]:
#attr_names = sorted(set([n for (n, _) in hmap_original.keys()]))
attr_names

In [ ]:
def get_sorting(n):
    attr_names_sorting = {
     'DTD': -1,
     'GuidedBP': -1,
     'SmoothGrad': -1,
     '$\\alpha=1, \\beta=0$-LRP': 1,
     '$\\alpha=2, \\beta=1$-LRP': 2,
     '$\\alpha=100, \\beta=99$-LRP': 3,
    }
    if n in attr_names_sorting:
        return attr_names_sorting[n]
    elif "epsilon" in n:
        return 0
    elif "cmp-LRP" in n:
        return 10
    else:
        return 5
    
attr_names = sorted(attr_names, key=lambda x: (get_sorting(x), x))
attr_names

In [ ]:
ssim = OrderedDict()
l2_random_weights = OrderedDict()

last_idx = len(model.layers)
for (name, img_idx, layer_idx), heatmap in tqdm.tqdm_notebook(hmap_random_weights.items()):
    original_heatmap = hmap_original[(name, img_idx)]
    postprocess = hmap_postprocesisng[name]
    original_heatmap = postprocess(original_heatmap)
    heatmap = postprocess(heatmap)
    ssim[(name, img_idx, layer_idx)] = ssim_flipped(heatmap, original_heatmap)
    l2_random_weights[(name, img_idx, layer_idx)] = l2_flipped(heatmap, original_heatmap)

In [ ]:
ssim_random_target = OrderedDict()
for (name, img_idx), (_, hmap_random) in tqdm.tqdm_notebook(hmap_random_target.items()):
    if name not in ssim_random_target:
        ssim_random_target[name] = []
        
    postprocess = hmap_postprocesisng[name]
    
    hmap = hmap_original[name, img_idx]
    original_heatmap = postprocess(original_heatmap)
    hmap = postprocess(hmap)
    hmap_random = postprocess(hmap_random)
    ssim_random_target[name].append(
        ssim_flipped(hmap, hmap_random))

In [ ]:
with sns.axes_style('whitegrid', {"axes.grid": True, 'font.family': 'serif'}):
    fig, ax = plt.subplots(1, 1, figsize=(4, 3.0), squeeze=True)
    mean_ssim = [np.mean(s) for s in ssim_random_target.values()]
    
    names = ssim_random_target.keys()
    bars = ax.bar(names, mean_ssim, 
           color=[linestyles[name]['color'] for name in attr_names])
    
    xlabels = ssim_random_target.keys()
    ax.set_ylabel('SSIM')
    ax.set_xticks(np.arange(len(xlabels)))
    ax.set_xticklabels(xlabels, rotation=90)
    
    
    fig.savefig('check-random-logit.pdf',  bbox_inches='tight', pad_inches=0)
    display(IFrame('check-random-logit.pdf', 800, 500))

In [ ]:
ssim_random_target.keys()

In [ ]:
with sns.axes_style('ticks', {"axes.grid": True, 'font.family': 'serif'}):
    fig, ax = plt.subplots(1, 1, figsize=(4, 3.0), squeeze=True)
    
    
    xlabels = attr_names
    bars = ax.boxplot([ssim_random_target[n] for n in attr_names]) 
    ax.set_ylabel('SSIM')
    #ax.set_xticks(np.arange(len(xlabels)))
    ax.set_xticklabels(xlabels, rotation=90)
    ax.set_ylim(-0.05, 1.05)
    
    fig.savefig('check-random-logit-boxplot.pdf',  bbox_inches='tight', pad_inches=0)
    display(IFrame('check-random-logit-boxplot.pdf', 800, 500))

In [ ]:
ssim_reduce = 'median'
confidence_intervals = True
confidence_percentile = 99.5
with sns.axes_style("ticks", {"axes.grid": True, 'font.family': 'serif'}):
    # metrics = [('SSIM', ssim), ('MSE', l2_random_weights)]
    metrics = [('SSIM', ssim)]
    fig, axes = plt.subplots(1, len(metrics), figsize=(4.5 * len(metrics), 3.5), squeeze=False)
    axes = axes[0]
    for ax, (ylabel, metric) in zip(axes, metrics): 
        for name in attr_names:
            metric_per_layer = []
            
            lower_conf = []
            upper_conf = []
            for (_, layer_idx) in selected_layers[::-1]:
                metric_per_layer.append(
                    [metric[(name, img_idx, layer_idx)] for img_idx in selected_img_idxs]
                )
                if confidence_intervals:
                    vals = np.array(metric_per_layer[-1])
                    ridx = np.random.choice(len(vals), (10000, len(vals)), replace=True)
                    resample = vals[ridx]
                    stats = np.median(resample, 1)
                    lower_conf.append(np.percentile(stats, 100 - confidence_percentile))
                    upper_conf.append(np.percentile(stats, confidence_percentile))
                    
            metric_per_layer = np.array(metric_per_layer)
                
            if ssim_reduce == 'mean':
                ssims_reduced = metric_per_layer.mean(1)
            elif ssim_reduce == 'median':
                ssims_reduced = np.median(metric_per_layer, 1)
            
            ticks = np.arange(len(ssims_reduced))
            ax.plot(ticks, ssims_reduced, label=name, **linestyles[name])
            ax.fill_between(ticks, lower_conf, upper_conf, 
                            color=linestyles[name]['color'],
                            alpha=0.25
                           )
            #ax.plot(ticks, lower_conf, color=linestyles[name]['color'])
            #ax.plot(ticks, upper_conf, color=linestyles[name]['color'])
            
        xlabels = [layer_name for layer_name, _ in selected_layers[::-1]] 
        ax.set_ylim([0, 1.05])
        ax.set_xticks(np.arange(len(xlabels)))
        ax.set_xticklabels(xlabels, rotation=90)
        ax.set_ylabel(ylabel)
    axes[-1].legend(bbox_to_anchor=(1.0, 1.00))
    plt.savefig('check-random-weights.pdf',  bbox_inches='tight', pad_inches=0)
    plt.show()
    display(IFrame('check-random-weights.pdf', 800, 500))

In [ ]:
def plot_heatmap_grid(heatmaps, cols, row_labels=[], column_labels=[], fig_path=None):
    mpl.rcParams['font.family'] = 'serif'
    rows = len(heatmaps) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols, rows), squeeze=False)
    fontsize = 12
    plt.subplots_adjust(wspace=0.05, hspace=0.05, top=1, bottom=0, left=0, right=1)
    for label, ax in zip(row_labels, axes[:, 0]):
        ax.set_ylabel(label, fontsize=fontsize - 1, labelpad=55, rotation=0)
        
    print(axes.shape, column_labels, row_labels)
    for label, ax in zip(column_labels, axes[0, :]):
        ax.set_title(label, fontsize=fontsize - 1)
        
        
    for ax, heatmap in zip(axes.flatten(), heatmaps):
        ax.imshow(heatmap, cmap='seismic', vmin=-1, vmax=1)
        ax.set_xticks([])
        ax.set_yticks([])

    #plt.tight_layout()
    if fig_path is not None:
        plt.savefig(fig_path, bbox_inches='tight', pad_inches=0, dpi=150)

In [ ]:
analysers

In [ ]:
def normalize_neg(x):
    vmax = np.percentile(x, 99)
    return np.clip(x / vmax, -1, 1)
for model_name in model_names:
    hmap_plot = []
    rnd_layers = randomization_layers[model_name][::-1]
    attr_for_model = []
    for (attr_name, _, _, excludes, _) in analysers[:4]:
        if 'exclude_' + model_name in excludes:
            print(attr_name)
            continue
        #try:
        if True:
            postp = hmap_postprocesisng[attr_name]

            for img_idx in selected_img_idxs[:1]:
                hmap_plot.append(norm_image(val_images[0][0][0]))
                hmap_plot.append(normalize_neg(postp(hmap_original[model_name, attr_name, img_idx])))
                for layer_name in rnd_layers:
                    layer_idx = get_layer_idx_full(model_name, nice_layer_names, layer_name)
                    hmap_plot.append(normalize_neg(postp(hmap_random_weights[model_name, attr_name, img_idx, layer_idx])))
            attr_for_model.append(attr_name)
        # except KeyError as e:
        #     print(dir(e))
        #     print(e)
        #     pass
    
    plot_heatmap_grid(
        hmap_plot, 2+len(rnd_layers), row_labels=attr_for_model, 
        column_labels=['input', 'original'] + rnd_layers,
        fig_path='heatmap_grid.pdf'
    )

In [ ]:
display(IFrame('heatmap_grid.pdf', width=1000, height=600))

In [ ]:
list(hmap_original.keys())

In [ ]:
raise Exception("stop ipynb execution")

In [ ]:
list(ssim.keys())[0]

In [ ]:
method = 'DTD'
layer_idx = selected_layers[4][1]
ssim_per_layer = []
for img_idx in selected_img_idxs:
    ssim_per_layer.append(ssim[method, img_idx, layer_idx])
    
ssim_per_layer = np.array(ssim_per_layer)

In [ ]:
means = []
medians = []

ridx = np.random.choice(n_selected_imgs, (10000, n_selected_imgs), replace=True)

means = ssim_per_layer[ridx].mean(1)
median = np.median(ssim_per_layer[ridx], 1)
print(np.percentile(median, 99))
print(np.percentile(median, 1))

## Relevance w.r.t. Parameters


In [ ]:
def prepare_idx(neuron_selection, batch_size=1):
    neuron_selection = np.asarray(neuron_selection).flatten()
    if neuron_selection.size == 1:
        neuron_selection = np.repeat(neuron_selection, batch_size)

    # Add first axis indices for gather_nd
    neuron_selection = np.hstack(
        (np.arange(len(neuron_selection)).reshape((-1, 1)),
         neuron_selection.reshape((-1, 1)))
    )
    return neuron_selection

In [ ]:
rel_wrt_kernels = OrderedDict()
for name, innv_name, innv_kwargs in analysers:
    print(name)
    analyzer = create_analyzer(
        innv_name, model_wo_softmax, neuron_selection_mode="index", **innv_kwargs)
    analyzer.create_analyzer_model()
    try:
        analyzer_model = analyzer._analyzer_model
    except:
        continue
    input_image, input_idx = analyzer_model.inputs

    rel_mean = tf.reduce_sum(analyzer_model.outputs[0])
    rel_grad_w = tf.gradients(
        rel_mean,
        analyzer_model.weights,
    )
    sess = keras.backend.get_session()
    outs = sess.run(analyzer._analyzer_model.outputs + rel_grad_w, 
                    {input_image: ex_image, input_idx: prepare_idx(ex_target)})
    
    output = outs[0]
    rel_grad_w_np = outs[1:]
    print(len(rel_grad_w_np))
    for i, w in enumerate(analyzer_model.weights):
        if "kernel" in w.name:
            #print(i, w.name)
            rel_wrt_kernels[name, ex_image_idx, i, w.name.rstrip("/kernel:0")] = rel_grad_w_np[i]

    print(output.min(), output.max())
    plt.imshow(normalize(output[0]))

In [ ]:
for (name, _, _) in analysers:
    grads = []
    percentile = 99
    lower_percentile = []
    upper_percentile = []
    layer_names = []
    for (gname, _, idx, layer_name), grad in rel_wrt_kernels.items():
        if name == gname:
            grads.append(np.abs(grad).mean())
            lower_percentile.append(np.percentile(np.abs(grad), 100-percentile))
            upper_percentile.append(np.percentile(np.abs(grad), percentile))
            layer_names.append(layer_name)
            
    plt.title(name)
    plt.semilogy(grads)
    plt.semilogy(lower_percentile)
    plt.semilogy(upper_percentile)
    plt.ylim(1e2, 1e-16)
    plt.xticks(ticks=np.arange(len(rel_wrt_kernels)), labels=layer_names, rotation=90)
    plt.show()


In [ ]:
for (name, _, _) in analysers:
    grads = []
    percentile = 99
    lower_percentile = []
    upper_percentile = []
    layer_names = []
    layer_idxs = []
    
    n_layers = 0
    i = 0
    for ((gname, _, idx, layer_name), grad) in rel_wrt_kernels.items():
        if name == gname:
            grads.append(np.abs(grad).flatten())
            layer_idxs.append(i*np.ones_like(grads[-1]))
            layer_names.append(layer_name)
            i += 1
          
    if len(grads) == 0:
        continue
     
    layer_idxs = np.concatenate(layer_idxs)
    grads = np.concatenate(grads)
    print('.')
    
    idx = np.random.choice(len(grads), size=100000)
    print('.')
    
    plt.title(name)
    plt.scatter(layer_idxs[idx], grads[idx], marker='_', alpha=0.1)
    plt.yscale('log')
    plt.ylim([1000, 1e-8])
    plt.xticks(ticks=np.arange(len(layer_names)), labels=layer_names, rotation=90)
    plt.show()

In [ ]:
for (name, _, _) in analysers:
    grads = []
    layer_names = []
    for (gname, _, idx, layer_name), grad in rel_wrt_kernels.items():
        if name == gname:
            grads.append((np.abs(grad) == 0).sum())
            layer_names.append(layer_name)
            
    plt.title(name)
    plt.scatter(np.arange(len(grads)), grads)
    plt.yscale('log')
    plt.ylim(1e9, 1)
    plt.xticks(ticks=np.arange(len(grads)), labels=layer_names, rotation=90)
    plt.show()
